<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


# Final Project - Earth's Favorite Football Team
### By: Brian Stroh
### Date: 12/6/2019

In the late 1970's, the Dallas Cowboys were coined as "America's Team" (See https://en.wikipedia.org/wiki/America%27s_Team ). A lot has changed since the 70's and whereas the Cowboys are leading their division with a baffling record of 6-7, I believe they are no longer the most exciting organization in football.

The goal of this project will be to identify the modern day "America's Team". The criteria will be the most popular team as determined by number of tweets mentioning the NFL teams in the time frame of 12/6/2019 to 12/10/2019.

Limitations of the data analysis stem from the data collection process. In order to prevent the API from being overloaded or exploited, Twitter's Streaming API limits the number of tweets that can be extracted. Fortunately, all NFL teams have games this week, so the analysis will not be skewed from that perspective. However, the Cowboys and Bears game was a day before the tweets began to be collected, so there may be a slight under-representation of those two teams, as their most exciting moments of the week have passed.

In [1]:
import tweepy
import os
import csv
os.chdir('C:/Users/brian/Desktop/All/UWEC/DS710_Programming_For_Data_Science/GitHub/ds710fall2019finalproject/')

In [2]:
%run C:/Users/brian/Desktop/All/UWEC/DS710_Programming_For_Data_Science/GitHub/ds710fall2019finalproject/twitter_credentials.py
# this cell will evaluate silently 🙊, and not print anything.  
# This is desired, because a person with your keys can act as you on Twitter in literally every way 😟

In [3]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [4]:
#We create a subclass of tweepy.StreamListener to add a response to on_status

class PrintingStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        #Extract tweet text
        with open('text.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            #This block comes from Twitter's API documentation: http://docs.tweepy.org/en/latest/extended_tweets.html
            if hasattr(status, "retweeted_status"):  # Check if Retweet
                try:
                    writer.writerow(status.retweeted_status.extended_tweet["full_text"])
                except AttributeError:
                    writer.writerow(status.retweeted_status.text)
            else:
                try:
                    writer.writerow(status.extended_tweet["full_text"])
                except AttributeError:
                    writer.writerow(status.text)
        
        #Extract tweet place (if applicable)
        with open('place.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([status.place])
            
        #Extract full tweet information
        with open('full.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([status])
            
    #disconnect the stream if we receive an error message indicating we are overloading Twitter
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

Once we have created our subclass, we can set up our own Twitter stream.

In [5]:
#We create and authenticate an instance of our new ```PrintingStreamListener``` class

my_stream_listener = PrintingStreamListener()
my_stream = tweepy.Stream(auth = api.auth, listener=my_stream_listener, tweet_mode = 'extended')

In [6]:
#The try-except here will suppress the big ugly traceback error that comes when the code is interrupted
try:
    my_stream.filter(track=['NFL'])
except KeyboardInterrupt:
    pass

In [7]:
my_stream.disconnect()